## Cross Var Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import s3fs
import spacy

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from ast import literal_eval
import re
# nltk.download() #<<< one time effort

### Load and Process Data

In [3]:
bucket = 's3://cortex-ni-wc-claims'
file = 'WC_DeEscl_Data_Build_v2.csv'

df = pd.read_csv(f'{bucket}/{file}')
print('Data shape: ', df.shape)

/mnt/miniconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data shape:  (544529, 100)


In [4]:
df.head(10)

Year  Claim_ID  Claim_Num Claim_System_Entry_Dte  Report_dte  Injury_Dte  \
0  2013   2518406  413A69752             2013-01-02  2013-01-01  2012-12-31   
1  2013   2539906  390A93167             2013-01-11  2013-01-10  2013-01-09   
2  2013   2551806  949A85874             2013-01-17  2013-01-16  2013-01-14   
3  2013   2555306  14CA14539             2013-01-29  2013-01-28  2013-01-27   
4  2013   2555906  413A71888             2013-01-31  2013-01-29  2012-10-07   
5  2013   2557406  205A40639             2013-02-05  2013-02-03  2013-02-02   
6  2013   2558006  608A58238             2013-02-06  2013-02-05  2013-01-23   
7  2013   2558106  413A72486             2013-02-06  2013-02-05  2012-03-01   
8  2013   2558506  80DA84769             2013-02-07  2013-02-06  2013-01-29   
9  2013   2563306  608A59396             2013-02-22  2013-02-21  2013-02-20   

  Claim_Status_Cde Current_PMT_Cde Current_Controverted_I  \
0                F              07                    NaN   
1                F              01                    NaN   
2                F              0P                      Y   
3                F              0P                    NaN   
4                F              04                    NaN   
5                F              01                    NaN   
6                F              M1                    NaN   
7                F              04                    NaN   
8                F              05                    NaN   
9                F              M7                      Y   

   current_total_incurred  ...  PrimNOI_pam_11c  Occur_km_6c  Occur_km_5c  \
0                     0.0  ...                5            5            1   
1                  5747.0  ...                5            5            1   
2                     0.0  ...                5            5            1   
3                     0.0  ...                7            5            1   
4                   495.0  ...               10            5            1   
5                     0.0  ...                8            5            1   
6                   397.0  ...                7            5            1   
7                   258.0  ...               10            4            2   
8                     0.0  ...                5            4            2   
9                   756.0  ...                7            4            2   

  Occur_km_4c Occur_km_3c  Occur_pam_6c Occur_pam_5c Occur_pam_4c  \
0           1           1             5            4            3   
1           1           1             5            4            3   
2           1           1             6            5            4   
3           1           1             5            4            3   
4           1           1             6            5            4   
5           1           1             5            4            3   
6           1           1             6            5            4   
7           3           3             3            3            1   
8           3           3             3            3            1   
9           3           3             4            3            1   

   Occur_pam_3c folds  
0             2     5  
1             2     4  
2             3     4  
3             2     4  
4             3     4  
5             2     1  
6             3     3  
7             1     2  
8             1     4  
9             1     4  

[10 rows x 100 columns]

In [5]:
list(df.columns.values)

['Year',
 'Claim_ID',
 'Claim_Num',
 'Claim_System_Entry_Dte',
 'Report_dte',
 'Injury_Dte',
 'Claim_Status_Cde',
 'Current_PMT_Cde',
 'Current_Controverted_I',
 'current_total_incurred',
 'Indemnity_Incurred',
 'Medical_Incurred',
 'Expense_Incurred',
 'Account_Name',
 'Top Account Name',
 'Lost_Time_Cde',
 'Jurisdiction_State_Cde',
 'Intake_Method',
 'Primary_Nature_Of_Injury_Cde',
 'Secondary_Nature_Of_Injury_Cde',
 'Primary_Part_Of_Body_Cde',
 'Secondary_Part_Of_Body_Cde',
 'Initial_Trtm_Cde',
 'Occurrence_Cde',
 'Claimant_Age_Qty',
 'Gender_Cde',
 'Tenure',
 'Report_Lag',
 'Avg_Wkly_Wge_Amt',
 'Runoff/Not Runoff',
 'Cust_Postal_Cde',
 'Claimant_Postal_Cde',
 'Occupation_Txt',
 'Accident_Note_Text',
 'Injury_Note_Text',
 'NAICS_Cde',
 'Segment_Today',
 'Dataset',
 'CRC_Rating_Plan_Code',
 'Initial_Seg',
 'InjurytoReport',
 'ReporttoEntry',
 'Prim_POB_Category_Cde',
 'Prim_POB_Dsc',
 'Prim_POB_Category_Dsc',
 'Scd_POB_Category_Cde',
 'Scd_POB_Dsc',
 'Scd_POB_Category_Dsc',
 'Occur_D

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
eng_stopwords = set(stopwords.words('english')) 
extra_stopwords = set(['left', 'right', 'employee', 'emp', 'ee', 'associate', 'iw']) 
STOPWORDS = set.union(eng_stopwords, extra_stopwords)

def text_prepare(text):
    """
        input: a string
        
        return: cleaned string
    """
    text = text.lower() # lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    word_tokens = word_tokenize(text) 
    word_tokens = [w for w in word_tokens if not w in STOPWORDS] # delete stopwords from text
  
    stemmer = nltk.stem.WordNetLemmatizer()
    for i in range(len(word_tokens)):
        # stemming + spelling correction
        
#          word_tokens[i] = token.lemma_( word_tokens[i])
        word_tokens[i] = stemmer.lemmatize(word_tokens[i])
#         word_tokens[i] = stemmer.lemmatize(spell(word_tokens[i]))
        
    text = " ".join(word_tokens)
    return text

In [7]:
# process text data
import time
start_time = time.time()

# fill missing notes
df['Accident_Note_Text'] = df['Accident_Note_Text'].fillna('unknown')
df['Injury_Note_Text'] = df['Injury_Note_Text'].fillna('unknown')

df['Accident_Note_Text'] = df['Accident_Note_Text'].apply( text_prepare)
df['Injury_Note_Text'] = df['Injury_Note_Text'].apply(text_prepare)

print("--- %s seconds ---" % (time.time() - start_time))

--- 175.0889914035797 seconds ---


In [8]:
df['Accident_Note_Text'].head(10)

0    assoicate injuring restriction said keyboard drop
1    working lightbulb aisle rochelle noticed ache ...
2                             causeunspecified medical
3    walked saying couldnt see walked eye station w...
4                  josh taking item microwave fell arm
5    dropped home depot knife break room bent pick ...
6                                              unknown
7                                state feel pain wrist
8                                      jerked arm hard
9                      felt light headed faint lifting
Name: Accident_Note_Text, dtype: object

In [9]:
df['Injury_Note_Text'].head(10)

0    unknown
1    unknown
2    unknown
3    unknown
4    unknown
5    unknown
6    unknown
7    unknown
8    unknown
9    unknown
Name: Injury_Note_Text, dtype: object

In [10]:
# creating the feature matrix 
from sklearn.feature_extraction.text import TfidfVectorizer
import time

start_time = time.time()

acct_vectorizer1 = TfidfVectorizer(max_features=1000)
inj_vectorizer1 = TfidfVectorizer(max_features=1000)
acct_tfidf1 = acct_vectorizer1.fit_transform(df['Accident_Note_Text']).toarray()
inj_tfidf1 = inj_vectorizer1.fit_transform(df['Injury_Note_Text']).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 14.209887742996216 seconds ---


In [11]:
start_time = time.time()

acct_vectorizer2 = TfidfVectorizer(max_features=1000,
                                  ngram_range=(2, 2)) # look at two gram phrase only
inj_vectorizer2 = TfidfVectorizer(max_features=1000,
                                 ngram_range=(2, 2))
acct_tfidf2 = acct_vectorizer2.fit_transform(df['Accident_Note_Text']).toarray()
inj_tfidf2 = inj_vectorizer2.fit_transform(df['Injury_Note_Text']).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 19.490063190460205 seconds ---


In [12]:
acct_features1 = acct_vectorizer1.get_feature_names()
acct_features2 = acct_vectorizer2.get_feature_names()

inj_features1 = inj_vectorizer1.get_feature_names()
inj_features2 = inj_vectorizer2.get_feature_names()

### Cross Variable Analysis

Based on the results from the tfidf anlysis by target group from EDA_tfidf.ipynb, several key words are selected to perform addtional analysis:
 - Parking lot: may indicate a more specific type of slip and fall
 - Pallet: certain type of injury caused by pallet may be less severe
 - Allergic: more specific of injury cause which can lead to various syptom 
 - Stroke seizure: may indicate claims that will be declined 

In [13]:
def seg_analysis(df, target, seg, group,  metrics):
    """
        input: 
        df = dataframe;
        target = response var, a list;
        seg = segmentation var, a string;
        group = group by criteria, a list;
        metrics = aggregation metrics, a list 
        
        return: summary dataframe
    """
#     seg = list(seg)
    target = list(target)
    group = list(group)
    
    col = target + group
    
    tb1 = df[col].groupby(group).agg(metrics)
#     tb2 = df[col].groupby(group).agg(metrics)
    tb2 = df[df[seg] == 1][col].groupby(group).agg(metrics)
    
    summary = pd.merge(tb1, tb2, on = group, how = 'outer')
    return summary

#### Parking Lot Analysis

In [14]:
prklot_idx = acct_tfidf2[:,acct_features2.index('parking lot')]. nonzero()

In [15]:
# overall level
df[[ 'Claimant_Age_Qty',
     'current_total_incurred',
     'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred   deEscInd3000
count     544529.000000            5.445290e+05  544529.000000
mean          40.998786            1.420452e+04       0.729581
std           13.777082            7.715754e+04       0.444177
min           15.000000            0.000000e+00       0.000000
25%           29.000000            0.000000e+00       0.000000
50%           41.000000            4.740000e+02       1.000000
75%           52.000000            3.673000e+03       1.000000
max           80.000000            1.571921e+07       1.000000

In [16]:
#rows with "parking lot"
df.iloc[prklot_idx][[ 'Claimant_Age_Qty',
                   'current_total_incurred',
                   'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred  deEscInd3000
count       5802.000000            5.802000e+03   5802.000000
mean          45.209928            1.332329e+04      0.723371
std           13.807055            6.010355e+04      0.447370
min           16.000000            0.000000e+00      0.000000
25%           34.000000            0.000000e+00      0.000000
50%           47.000000            3.880000e+02      1.000000
75%           56.000000            3.858750e+03      1.000000
max           80.000000            2.874258e+06      1.000000

From the overall level, claims with "parking lot" does not seem to differentiate from the rest

In [17]:
df.iloc[prklot_idx][['Intake_Method',
                     'Gender_Cde',
                     'Prim_POB_Dsc',
                     'Prim_POB_Category_Dsc','Scd_POB_Dsc',
                     'Scd_POB_Category_Dsc',
                     'Occur_Dsc',
                     'Prim_NOI_Dsc']].describe()

Intake_Method Gender_Cde Prim_POB_Dsc  \
count                       5802       5802         5802   
unique                         8          2           52   
top     Service Center Telephone          F         Knee   
freq                        2414       3316          890   

       Prim_POB_Category_Dsc Scd_POB_Dsc Scd_POB_Category_Dsc  \
count                   5802        5802                 5802   
unique                     7          42                    8   
top        LOWER EXTREMITIES     missing              missing   
freq                    2234        4072                 4072   

                            Occur_Dsc Prim_NOI_Dsc  
count                            5802         5802  
unique                             56           34  
top     fall/slip/trip on ice or snow       Strain  
freq                             2080         1814

In [24]:
df['prklot_idx'] = np.where(acct_tfidf2[:,acct_features2.index('parking lot')] > 0, 1, 0)

In [65]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'prklot_idx', 
             ['Occur_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count'), 'Occur_Dsc'], ascending = False).head(60)

deEscInd3000_x            \
                                                            count      mean   
Occur_Dsc                                                                     
fall/slip/trip on ice or snow                               10268  0.632743   
fall/slip/trip on same level                                25274  0.656089   
fall, slip or trip, not otherwise classified                19272  0.657482   
strain or injury by not otherwise specified                 71699  0.729940   
fall/slip/trip from liquid or grease spills                  9284  0.680741   
Injury from Motor vehicle                                    2310  0.507359   
person in an act of a crime                                  6091  0.807749   
Collision or sideswipe with another vehicle                 10885  0.642076   
fall/slip/trip from different level/ elevation              10971  0.546714   
Miscellaneous                                               12805  0.835143   
slip or trip, did not fall                                   4320  0.639352   
other than physical cause of injury                         19313  0.878061   
Other - miscellaneous, not otherwise classified             13137  0.788917   
Strain or injury by pushing or pulling                      19756  0.635351   
Cumulative, not otherwise classified                        12892  0.774899   
Injury from Falling or flying object                        27555  0.791435   
Motor vehicle, not otherwise classified                      4144  0.616554   
Strain or injury by lifting or carrying                     57549  0.693913   
fall/slip/trip into openings                                 1530  0.624837   
Striking Against or Stepping On, Not Otherwise ...           7206  0.798085   
Strain or injury by twisting                                 6535  0.665340   
Contact with animal or insect                                8146  0.929168   
Striking Against or Stepping On Stationary Object           11148  0.819878   
Fall/slip/trip on stairs                                     7812  0.639017   
cut, puncture, scrape not otherwise classified              12991  0.910938   
Collision with a fixed object                                1171  0.602050   
Injury from Object Being Lifted or Handled                  11167  0.785708   
Striking Against or Stepping On Sharp Object                 1412  0.924929   
Absorption, ingestion, or inhalation not otherw...          11084  0.871527   
Injury from Object Handled by Others                         2931  0.778915   
Injury by Person - fellow worker/patient (not i...           8724  0.775906   
foreign matter (body) in eye(s)                              5427  0.964253   
caught in/under/between object handled                       5652  0.725584   
caught in/under/between not otherwise classified             5332  0.707802   
Strain or injury by repetitive motion                       25881  0.601870   
Strain or injury by holding or carrying                      6296  0.647872   
Strain or injury by jumping or leaping                       1130  0.602655   
Injury from Hand Tool or Machine in Use                      2899  0.696447   
Cut/puncture/scrape by hand tool, utensil, not ...           9532  0.927193   
Burn or Scald by Temperature extremes                         932  0.839056   
Vehicle upset (overturned)                                   1196  0.418896   
Strain or injury by using tool or machinery                  3430  0.684548   
Cut/puncture/scrape by object being lifted or h...          10176  0.876572   
fall/slip/trip from ladder or scaffolding                    5746  0.504525   
Strain or injury by reaching                                 4841  0.673621   
Contact with electric current                                 922  0.718004   
Striking Against or Stepping On Object Being Li...           1070  0.782243   
Injury from Moving Parts of Machine                           987  0.587639   
Cut/puncture/scrape by

**Comments**: claims with 'parking lot' tend to have lower cost for slip and fall generally and any vehicle related injury, which makes sense considering the speed 

In [67]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'prklot_idx', 
             ['age_5grp'],
             ['count', 'mean'])

deEscInd3000_x           deEscInd3000_y          
                   count      mean          count      mean
age_5grp                                                   
001:15-27         118284  0.849557            843  0.860024
002:28-36         106497  0.753580            860  0.798837
003:37-46         112856  0.697039           1131  0.729443
004:47-54          99954  0.664376           1280  0.672656
005:55-80         106938  0.668266           1688  0.651066

**Comments**: claims with 'parking lot' tend to have lower cost for age below 50

In [69]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'prklot_idx', 
             ['Prim_POB_Category_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x           deEscInd3000_y          
                               count      mean          count      mean
Prim_POB_Category_Dsc                                                  
LOWER EXTREMITIES             114223  0.701242           2234  0.718890
UPPER EXTREMITIES             182661  0.718801           1245  0.666667
TRUNK                         126858  0.711457           1018  0.768173
HEAD                           71762  0.816937            774  0.729974
MISCELLANEOUS                  19085  0.866440            217  0.917051
MULTIPLE BODY PARTS            15912  0.710533            193  0.694301
NECK                           14028  0.653122            121  0.669421

**Comments**: claims with 'parking lot' tend to higher cost if head is injured and have lower cost if trunk or miscellaneous body part is injured

In [71]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'prklot_idx', 
             ['Prim_POB_Category_Dsc', 'age_5grp'],
             ['count', 'mean'])

deEscInd3000_x           deEscInd3000_y  \
                                         count      mean          count   
Prim_POB_Category_Dsc age_5grp                                            
HEAD                  001:15-27          17939  0.889905            121   
                      002:28-36          13397  0.844592             94   
                      003:37-46          13202  0.816316            114   
                      004:47-54          11895  0.778731            173   
                      005:55-80          15329  0.737556            272   
LOWER EXTREMITIES     001:15-27          24322  0.825837            326   
                      002:28-36          21678  0.723037            339   
                      003:37-46          23100  0.669827            478   
                      004:47-54          21546  0.637241            504   
                      005:55-80          23577  0.641939            587   
MISCELLANEOUS         001:15-27           4852  0.929308             51   
                      002:28-36           3668  0.867503             44   
                      003:37-46           3652  0.835706             42   
                      004:47-54           3267  0.825222             34   
                      005:55-80           3646  0.849424             46   
MULTIPLE BODY PARTS   001:15-27           2948  0.821574             28   
                      002:28-36           3036  0.733202             29   
                      003:37-46           3476  0.691312             30   
                      004:47-54           3088  0.655440             38   
                      005:55-80           3364  0.663199             68   
NECK                  001:15-27           2102  0.789724             15   
                      002:28-36           3053  0.691451             29   
                      003:37-46           3270  0.613761             22   
                      004:47-54           2951  0.600474             26   
                      005:55-80           2652  0.607843             29   
TRUNK                 001:15-27          24481  0.826600            151   
                      002:28-36          26180  0.722422            159   
                      003:37-46          28320  0.674718            209   
                      004:47-54          23552  0.663128            227   
                      005:55-80          24325  0.673340            272   
UPPER EXTREMITIES     001:15-27          41640  0.855235            151   
                      002:28-36          35485  0.756179            166   
                      003:37-46          37836  0.683080            236   
                      004:47-54          33655  0.633011            278   
                      005:55-80          34045  0.637480            414   

                                           
                                     mean  
Prim_POB_Category_Dsc age_5grp             
HEAD                  001:15-27  0.834711  
                      002:28-36  0.840426  
                      003:37-46  0.789474  
                      004:47-54  0.664740  
                      005:55-80  0.661765  
LOWER EXTREMITIES     001:15-27  0.858896  
                      002:28-36  0.772861  
                      003:37-46  0.713389  
                      004:47-54  0.680556  
                      005:55-80  0.647359  
MISCELLANEOUS         001:15-27  0.941176  
                      002:28-36  0.909091  
                      003:37-46  0.904762  
                      004:47-54  0.911765  
                      005:55-80  0.913043  
MULTIPLE BODY PARTS   001:15-27  0.750000  
                      002:28-36  0.724138  
                      003:37-46  0.833333  
                      004:47-54  0.552632  
                      005:55-80  0.676471  
NECK                  001:15-27  0.800000  
                      002:28-36  0.655172  
                      003:37-46  0.545455  
                      004:47-54  0.73

**Comments**: Head injury seems to cost more and trunck injury seems to cost less regardless of age. Lower and upper extermities vary by age.

In [72]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'prklot_idx', 
             ['Prim_POB_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x            \
                                                count      mean   
Prim_POB_Dsc                                                      
Knee                                            41048  0.632455   
Ankle                                           21831  0.701754   
Low back (lumbar or lumbo-sacral area)          80770  0.696694   
Head                                            34562  0.785834   
Shoulder                                        38111  0.568550   
Foot                                            21218  0.768593   
Wrist                                           26294  0.716779   
Hand                                            31927  0.791399   
Multiple body parts                             14294  0.700014   
Lower leg                                       13781  0.743633   
Hip                                              5864  0.668656   
Buttocks                                         2612  0.813936   
Elbow                                           11159  0.675060   
Lower arm                                       17672  0.778293   
No Physical Injury                              14012  0.880816   
Neck                                            12570  0.644710   
Insufficient Information                         5014  0.825888   
Facial soft tissue                               7349  0.851953   
Chest, ribs or sternum                           9030  0.787375   
Brain                                            7005  0.861956   
Finger(s)                                       34230  0.808706   
Abdomen                                         14297  0.643492   
Upper back (thoracic area)                       6340  0.742587   
Thigh                                            3223  0.739684   
Upper arm, clavicle or scapula                   5187  0.584924   
Eye(s)                                          12552  0.947100   
Sacrum or coccyx                                  581  0.736661   
Wrist(s) & Hand(s)                               4928  0.609781   
Lower extremities                                1299  0.646651   
Heart                                            3371  0.933254   
Thumb                                           11248  0.824769   
Nose                                             1930  0.795337   
Toe(s)                                           4020  0.863682   
Lungs                                            6752  0.782879   
Upper extremities                                1905  0.645669   
Trunk                                             852  0.733568   
Mouth                                            1437  0.860125   
Neck soft tissue                                  828  0.692029   
Great Toe                                        1939  0.847860   
Facial bones                                      890  0.625843   
Body Systems and Multiple Body Systems           1618  0.803461   
Teeth                                            1306  0.696018   
Lumbar and/or Sacral Vertebrae                    178  0.561798   
Pelvis                                            868  0.550691   
Ear(s)                                           4478  0.638901   
Skull                                             253  0.711462   
Trachea                                           224  0.892857   
Larynx                                            198  0.858586   
Artificial Appliance                               59  0.898305   
Disc in low or middle back                        272  0.496324   
Internal organs                                   720  0.719444   
Spinal cord (neck area)                            64  0.640625   
Disc in neck                                       70  0.485714   
Spinal cord (low or middle back area)             215  0.567442   
Vertebrae in neck                                  74  0.540541   

                                       deEscInd3000_y            
                                                count      mean  
Prim_POB_Dsc        

**Comments**: Knee, lower back injury seems to cost less in claims with 'parking lot', but head injury seems to cost much more

In [73]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'prklot_idx', 
             ['Prim_NOI_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x            \
                                                            count      mean   
Prim_NOI_Dsc                                                                  
Strain                                                     211564  0.679355   
Contusion                                                   84735  0.801227   
Fracture                                                    24608  0.419660   
Sprain                                                      32230  0.665281   
Laceration                                                  52728  0.865897   
Inflammation                                                21784  0.792325   
Injury                                                       9762  0.782217   
Syncope                                                     12590  0.943288   
Concussion                                                   5585  0.654790   
Multiple Physical Injuries                                   8576  0.714552   
Puncture                                                    13835  0.912685   
Mental stress                                                6195  0.790476   
Foreign body                                                 8312  0.955245   
Dislocation                                                  2289  0.471822   
Multiple Injuries                                            1480  0.658784   
Myocardial infarction (heart attack)                         2202  0.943688   
Dermatitis                                                   4586  0.960096   
Hernia                                                       6462  0.472609   
Vascular                                                      670  0.883582   
Burn                                                         7523  0.844876   
Crushing                                                     2905  0.575215   
Contagious diseases                                           789  0.936629   
Angina pectoris (chest pain)                                 1623  0.927911   
Respiratory Disorders                                        2269  0.840899   
Heat prostration (heat stroke)                                282  0.833333   
Electric Shock                                                711  0.755274   
No Physical Injury                                             59  0.898305   
Rupture                                                       229  0.572052   
Infection                                                    1875  0.820267   
Enucleation (surgical removal of a tumor, eye, ...             12  0.833333   
Cumulative injury                                            4658  0.535208   
Severence                                                     403  0.471464   
Cancer                                                        160  0.568750   
Traumatic hearing loss                                        974  0.798768   
Aids                                                            6  0.833333   
Amputation                                                   1000  0.202000   
Asbestosis (lung disease from breathing asbestos)            1881  0.699096   
Asphyxiation                                                   88  0.909091   
Black Lung disease                                            212  0.698113   
Byssinosis (lung disease from breathing cotton,...              3  0.000000   
Carpal tunnel syndrome                                       2723  0.522585   
Chemical poisoning                                            664  0.823795   
Exposure to dust                                              205  0.751220   
Freezing                                                       98  0.877551   
General Poisoning                                             145  0.882759   
Loss of hearing                                              2135  0.570023   
Mental disorder                                                52  0.769231   
Metal poisoning                                                32  0.343750   
Occupational disease  

**Comments:** Strain and sprain tend to cost less in claims with 'parking lot' 

#### Pallet Analysis

In [75]:
pallet_idx = acct_tfidf1[:,acct_features1.index('pallet')]. nonzero()

In [76]:
# overall level
df[[ 'Claimant_Age_Qty',
     'current_total_incurred',
     'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred   deEscInd3000
count     544529.000000            5.445290e+05  544529.000000
mean          40.998786            1.420452e+04       0.729581
std           13.777082            7.715754e+04       0.444177
min           15.000000            0.000000e+00       0.000000
25%           29.000000            0.000000e+00       0.000000
50%           41.000000            4.740000e+02       1.000000
75%           52.000000            3.673000e+03       1.000000
max           80.000000            1.571921e+07       1.000000

In [77]:
#rows with "pallet"
df.iloc[pallet_idx][['Claimant_Age_Qty',
                   'current_total_incurred',
                   'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred  deEscInd3000
count      17284.000000            1.728400e+04  17284.000000
mean          39.980213            1.160975e+04      0.755728
std           13.656851            5.998432e+04      0.429667
min           15.000000            0.000000e+00      0.000000
25%           28.000000            0.000000e+00      1.000000
50%           40.000000            5.090000e+02      1.000000
75%           51.000000            2.830000e+03      1.000000
max           80.000000            5.289320e+06      1.000000

**Comments:** On an overall level claims with 'pallet' seem to have lower cost especially in the tail

In [78]:
df.iloc[pallet_idx][['Intake_Method',
                     'Gender_Cde',
                     'Prim_POB_Dsc',
                     'Prim_POB_Category_Dsc','Scd_POB_Dsc',
                     'Scd_POB_Category_Dsc',
                     'Occur_Dsc',
                     'Prim_NOI_Dsc']].describe()

Intake_Method Gender_Cde                            Prim_POB_Dsc  \
count          17284      17284                                   17284   
unique             6          2                                      55   
top              EDI          M  Low back (lumbar or lumbo-sacral area)   
freq           10416      13026                                    3265   

       Prim_POB_Category_Dsc Scd_POB_Dsc Scd_POB_Category_Dsc  \
count                  17284       17284                17284   
unique                     7          49                    8   
top        LOWER EXTREMITIES     missing              missing   
freq                    6064       15683                15683   

                                      Occur_Dsc Prim_NOI_Dsc  
count                                     17284        17284  
unique                                       60           36  
top     Strain or injury by lifting or carrying       Strain  
freq                                       2867         7201

In [79]:
df['pallet_idx'] = np.where(acct_tfidf1[:,acct_features1.index('pallet')] > 0, 1, 0)

In [80]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['Occur_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count'), 'Occur_Dsc'], ascending = False).head(60)

deEscInd3000_x            \
                                                            count      mean   
Occur_Dsc                                                                     
Strain or injury by lifting or carrying                     57549  0.693913   
strain or injury by not otherwise specified                 71699  0.729940   
Injury from Falling or flying object                        27555  0.791435   
Strain or injury by pushing or pulling                      19756  0.635351   
fall/slip/trip on same level                                25274  0.656089   
fall, slip or trip, not otherwise classified                19272  0.657482   
Miscellaneous                                               12805  0.835143   
Injury from Object Being Lifted or Handled                  11167  0.785708   
cut, puncture, scrape not otherwise classified              12991  0.910938   
caught in/under/between not otherwise classified             5332  0.707802   
Striking Against or Stepping On Stationary Object           11148  0.819878   
Striking Against or Stepping On, Not Otherwise ...           7206  0.798085   
caught in/under/between object handled                       5652  0.725584   
Cut/puncture/scrape by object being lifted or h...          10176  0.876572   
Strain or injury by twisting                                 6535  0.665340   
fall/slip/trip from different level/ elevation              10971  0.546714   
Injury from Hand Tool or Machine in Use                      2899  0.696447   
Injury from Object Handled by Others                         2931  0.778915   
Striking Against or Stepping On Sharp Object                 1412  0.924929   
Cut/puncture/scrape by hand tool, utensil, not ...           9532  0.927193   
Strain or injury by holding or carrying                      6296  0.647872   
Strain or injury by reaching                                 4841  0.673621   
Strain or injury by using tool or machinery                  3430  0.684548   
Strain or injury by repetitive motion                       25881  0.601870   
Other - miscellaneous, not otherwise classified             13137  0.788917   
caught in/under/between machine or machinery                 3140  0.484076   
foreign matter (body) in eye(s)                              5427  0.964253   
slip or trip, did not fall                                   4320  0.639352   
fall/slip/trip from liquid or grease spills                  9284  0.680741   
fall/slip/trip on ice or snow                               10268  0.632743   
Injury from Moving Parts of Machine                           987  0.587639   
Striking Against or Stepping On Object Being Li...           1070  0.782243   
Absorption, ingestion, or inhalation not otherw...          11084  0.871527   
Cumulative, not otherwise classified                        12892  0.774899   
other than physical cause of injury                         19313  0.878061   
Contact with animal or insect                                8146  0.929168   
Strain or injury by jumping or leaping                       1130  0.602655   
Injury from Motor vehicle                                    2310  0.507359   
Cut/puncture/scrape by powered hand tool, appli...           3349  0.673634   
fall/slip/trip into openings                                 1530  0.624837   
fall/slip/trip from ladder or scaffolding                    5746  0.504525   
Injury by Person - fellow worker/patient (not i...           8724  0.775906   
Strain or injury by wielding or throwing                      430  0.627907   
rubbed or abraded by repetitive motion                       1372  0.605685   
person in an act of a crime                                  6091  0.807749   
Burn or scald by Chemicals                                   1457  0.871654   
Fall/slip/trip on stairs                                     7812  0.639017   
Cut/puncture/scrape by broken glass                          2665  0.874296   
caught in/under/betwee

**Comments**: For claims with 'pallet', "Strain or injury by lifting or carrying", "strain or injury by not otherwise specified", "Injury from Falling or flying object", "fall, slip or trip, not otherwise classified", "cut, puncture, scrape not otherwise classified", tend to have lower cost

In [81]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['age_5grp'],
             ['count', 'mean'])

deEscInd3000_x           deEscInd3000_y          
                   count      mean          count      mean
age_5grp                                                   
001:15-27         118284  0.849557           4236  0.874646
002:28-36         106497  0.753580           3359  0.774338
003:37-46         112856  0.697039           3628  0.716648
004:47-54          99954  0.664376           3046  0.695995
005:55-80         106938  0.668266           3015  0.675290

**Comments**: Trend found in all ages, more obvious in younger age

In [83]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['Prim_POB_Category_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x           deEscInd3000_y          
                               count      mean          count      mean
Prim_POB_Category_Dsc                                                  
LOWER EXTREMITIES             114223  0.701242           6064  0.773252
UPPER EXTREMITIES             182661  0.718801           4717  0.736485
TRUNK                         126858  0.711457           4523  0.728499
HEAD                           71762  0.816937           1553  0.848680
NECK                           14028  0.653122            239  0.631799
MULTIPLE BODY PARTS            15912  0.710533             96  0.604167
MISCELLANEOUS                  19085  0.866440             92  0.836957

**Comments:** For claims with 'pallet', lower extermities have a strong lower cost pattern 

In [85]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['Prim_POB_Category_Dsc', 'age_5grp'],
             ['count', 'mean'])

deEscInd3000_x           deEscInd3000_y  \
                                         count      mean          count   
Prim_POB_Category_Dsc age_5grp                                            
HEAD                  001:15-27          17939  0.889905            371   
                      002:28-36          13397  0.844592            270   
                      003:37-46          13202  0.816316            303   
                      004:47-54          11895  0.778731            254   
                      005:55-80          15329  0.737556            355   
LOWER EXTREMITIES     001:15-27          24322  0.825837           1677   
                      002:28-36          21678  0.723037           1180   
                      003:37-46          23100  0.669827           1242   
                      004:47-54          21546  0.637241            970   
                      005:55-80          23577  0.641939            995   
MISCELLANEOUS         001:15-27           4852  0.929308             23   
                      002:28-36           3668  0.867503             12   
                      003:37-46           3652  0.835706             20   
                      004:47-54           3267  0.825222             20   
                      005:55-80           3646  0.849424             17   
MULTIPLE BODY PARTS   001:15-27           2948  0.821574             13   
                      002:28-36           3036  0.733202             17   
                      003:37-46           3476  0.691312             26   
                      004:47-54           3088  0.655440             19   
                      005:55-80           3364  0.663199             21   
NECK                  001:15-27           2102  0.789724             39   
                      002:28-36           3053  0.691451             55   
                      003:37-46           3270  0.613761             46   
                      004:47-54           2951  0.600474             53   
                      005:55-80           2652  0.607843             46   
TRUNK                 001:15-27          24481  0.826600            984   
                      002:28-36          26180  0.722422            957   
                      003:37-46          28320  0.674718           1012   
                      004:47-54          23552  0.663128            846   
                      005:55-80          24325  0.673340            724   
UPPER EXTREMITIES     001:15-27          41640  0.855235           1129   
                      002:28-36          35485  0.756179            868   
                      003:37-46          37836  0.683080            979   
                      004:47-54          33655  0.633011            884   
                      005:55-80          34045  0.637480            857   

                                           
                                     mean  
Prim_POB_Category_Dsc age_5grp             
HEAD                  001:15-27  0.927224  
                      002:28-36  0.881481  
                      003:37-46  0.864686  
                      004:47-54  0.807087  
                      005:55-80  0.757746  
LOWER EXTREMITIES     001:15-27  0.887895  
                      002:28-36  0.783898  
                      003:37-46  0.719002  
                      004:47-54  0.727835  
                      005:55-80  0.679397  
MISCELLANEOUS         001:15-27  0.913043  
                      002:28-36  0.833333  
                      003:37-46  1.000000  
                      004:47-54  0.850000  
                      005:55-80  0.529412  
MULTIPLE BODY PARTS   001:15-27  0.923077  
                      002:28-36  0.529412  
                      003:37-46  0.653846  
                      004:47-54  0.368421  
                      005:55-80  0.619048  
NECK                  001:15-27  0.871795  
                      002:28-36  0.654545  
                      003:37-46  0.543478  
                      004:47-54  0.52

**Comments:** Trend is generally consistent cross different age groups. No additional interaction detected.

In [86]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['Prim_POB_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x            \
                                                count      mean   
Prim_POB_Dsc                                                      
Low back (lumbar or lumbo-sacral area)          80770  0.696694   
Foot                                            21218  0.768593   
Knee                                            41048  0.632455   
Shoulder                                        38111  0.568550   
Ankle                                           21831  0.701754   
Head                                            34562  0.785834   
Finger(s)                                       34230  0.808706   
Lower leg                                       13781  0.743633   
Wrist                                           26294  0.716779   
Hand                                            31927  0.791399   
Abdomen                                         14297  0.643492   
Lower arm                                       17672  0.778293   
Toe(s)                                           4020  0.863682   
Elbow                                           11159  0.675060   
Eye(s)                                          12552  0.947100   
Thumb                                           11248  0.824769   
Chest, ribs or sternum                           9030  0.787375   
Upper back (thoracic area)                       6340  0.742587   
Neck                                            12570  0.644710   
Great Toe                                        1939  0.847860   
Hip                                              5864  0.668656   
Facial soft tissue                               7349  0.851953   
Buttocks                                         2612  0.813936   
Upper arm, clavicle or scapula                   5187  0.584924   
Multiple body parts                             14294  0.700014   
Thigh                                            3223  0.739684   
No Physical Injury                              14012  0.880816   
Nose                                             1930  0.795337   
Brain                                            7005  0.861956   
Insufficient Information                         5014  0.825888   
Wrist(s) & Hand(s)                               4928  0.609781   
Mouth                                            1437  0.860125   
Trunk                                             852  0.733568   
Lower extremities                                1299  0.646651   
Pelvis                                            868  0.550691   
Upper extremities                                1905  0.645669   
Ear(s)                                           4478  0.638901   
Facial bones                                      890  0.625843   
Lungs                                            6752  0.782879   
Teeth                                            1306  0.696018   
Heart                                            3371  0.933254   
Sacrum or coccyx                                  581  0.736661   
Neck soft tissue                                  828  0.692029   
Skull                                             253  0.711462   
Lumbar and/or Sacral Vertebrae                    178  0.561798   
Disc in low or middle back                        272  0.496324   
Internal organs                                   720  0.719444   
Body Systems and Multiple Body Systems           1618  0.803461   
Spinal cord (low or middle back area)             215  0.567442   
Larynx                                            198  0.858586   
Artificial Appliance                               59  0.898305   
Spinal cord (neck area)                            64  0.640625   
Disc in neck                                       70  0.485714   
Trachea                                           224  0.892857   
Vertebrae in neck                                  74  0.540541   

                                       deEscInd3000_y            
                                                count      mean  
Prim_POB_Dsc        

**Comments**: low back injur seemes to have lower cost

In [87]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['Prim_NOI_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False) 

deEscInd3000_x            \
                                                            count      mean   
Prim_NOI_Dsc                                                                  
Strain                                                     211564  0.679355   
Contusion                                                   84735  0.801227   
Laceration                                                  52728  0.865897   
Sprain                                                      32230  0.665281   
Fracture                                                    24608  0.419660   
Puncture                                                    13835  0.912685   
Inflammation                                                21784  0.792325   
Hernia                                                       6462  0.472609   
Foreign body                                                 8312  0.955245   
Crushing                                                     2905  0.575215   
Injury                                                       9762  0.782217   
Dislocation                                                  2289  0.471822   
Concussion                                                   5585  0.654790   
Syncope                                                     12590  0.943288   
Multiple Physical Injuries                                   8576  0.714552   
Infection                                                    1875  0.820267   
Dermatitis                                                   4586  0.960096   
Mental stress                                                6195  0.790476   
Angina pectoris (chest pain)                                 1623  0.927911   
Respiratory Disorders                                        2269  0.840899   
Traumatic hearing loss                                        974  0.798768   
Vision loss                                                   221  0.864253   
Multiple Injuries                                            1480  0.658784   
Amputation                                                   1000  0.202000   
Cumulative injury                                            4658  0.535208   
Severence                                                     403  0.471464   
Rupture                                                       229  0.572052   
Chemical poisoning                                            664  0.823795   
Burn                                                         7523  0.844876   
Loss of hearing                                              2135  0.570023   
Myocardial infarction (heart attack)                         2202  0.943688   
Electric Shock                                                711  0.755274   
Carpal tunnel syndrome                                       2723  0.522585   
Heat prostration (heat stroke)                                282  0.833333   
No Physical Injury                                             59  0.898305   
Contagious diseases                                           789  0.936629   
Aids                                                            6  0.833333   
Asbestosis (lung disease from breathing asbestos)            1881  0.699096   
Asphyxiation                                                   88  0.909091   
Black Lung disease                                            212  0.698113   
Byssinosis (lung disease from breathing cotton,...              3  0.000000   
Cancer                                                        160  0.568750   
Enucleation (surgical removal of a tumor, eye, ...             12  0.833333   
Exposure to dust                                              205  0.751220   
Freezing                                                       98  0.877551   
General Poisoning                                             145  0.882759   
Mental disorder                                                52  0.769231   
Metal poisoning                                                32  0.343750   
Occupational disease  

**Comments:** Contusion, laceration and puncture seem to have a lower cost for claims with "pallet". There is some moderate trend with Strain. We will break it down by age groups to take a closer look.

In [91]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'pallet_idx', 
             ['Prim_NOI_Dsc', 'age_5grp'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False) 

deEscInd3000_x  \
                                                                      count   
Prim_NOI_Dsc                                       age_5grp                   
Strain                                             003:37-46          47337   
                                                   001:15-27          39160   
                                                   002:28-36          42378   
                                                   004:47-54          41878   
Contusion                                          001:15-27          20652   
Strain                                             005:55-80          40811   
Contusion                                          005:55-80          18404   
                                                   003:37-46          15823   
                                                   002:28-36          15195   
                                                   004:47-54          14661   
Laceration                                         001:15-27          15973   
                                                   005:55-80           9151   
                                                   003:37-46           9349   
                                                   002:28-36          10650   
                                                   004:47-54           7605   
Sprain                                             003:37-46           7655   
Puncture                                           001:15-27           3835   
Sprain                                             004:47-54           6214   
                                                   002:28-36           6779   
                                                   001:15-27           6002   
                                                   005:55-80           5580   
Fracture                                           003:37-46           4875   
                                                   004:47-54           4873   
Puncture                                           002:28-36           3500   
Fracture                                           005:55-80           6264   
                                                   001:15-27           4272   
                                                   002:28-36           4324   
Inflammation                                       003:37-46           4703   
Puncture                                           003:37-46           2770   
Inflammation                                       001:15-27           4696   
...                                                                     ...   
No Physical Injury                                 005:55-80              5   
Occupational disease                               001:15-27             39   
                                                   002:28-36             60   
                                                   003:37-46             81   
                                                   004:47-54             67   
                                                   005:55-80             95   
Radiation                                          001:15-27              6   
                                                   002:28-36              8   
                                                   003:37-46              4   
                                                   004:47-54              2   
                                                   005:55-80              8   
Rupture                                            001:15-27             35   
                                                   002:28-36             29   
                                                   003:37-46             48   
Severence                                          002:28-36             89   
                                                   005:55-80             57   
Silicosis (lung disease from breathing silica d... 001:15-27              2   
                                

**Comments:** Trend to consistent cross age groups

In [88]:
df[["prklot_idx", "pallet_idx", 'deEscInd3000']]. groupby(["prklot_idx", "pallet_idx"]). agg(['count'])

deEscInd3000
                             count
prklot_idx pallet_idx             
0          0                521456
           1                 17271
1          0                  5789
           1                    13

**Comments:** The parking lot and pallet indicator have little correlation

#### Allergic Indicator

In [30]:
allrg_idx = acct_tfidf1[:,acct_features1.index('allergic')]. nonzero()

In [32]:
df.iloc[allrg_idx][[ 'Claimant_Age_Qty',
                   'current_total_incurred',
                   'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred  deEscInd3000
count  1541.000000       1541.000000             1541.000000 
mean   35.994160         1849.064244             0.948735    
std    12.939343         17709.576436            0.220610    
min    16.000000         0.000000                0.000000    
25%    25.000000         0.000000                1.000000    
50%    34.000000         0.000000                1.000000    
75%    45.000000         310.000000              1.000000    
max    74.000000         463282.000000           1.000000

**Comments**: Significant overall pattern

In [35]:
df['allrg_idx'] = np.where(acct_tfidf1[:,acct_features1.index('allergic')] > 0, 1, 0)

In [36]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'allrg_idx', 
             ['age_5grp'],
             ['count', 'mean'])

deEscInd3000_x           deEscInd3000_y          
                   count      mean          count      mean
age_5grp                                                   
001:15-27  118284         0.849557  516            0.976744
002:28-36  106497         0.753580  349            0.954155
003:37-46  112856         0.697039  326            0.932515
004:47-54  99954          0.664376  181            0.944751
005:55-80  106938         0.668266  169            0.887574

**Comments:** Consistent cross age groups

In [39]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'allrg_idx', 
             ['Prim_NOI_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False). head(30) 

deEscInd3000_x            \
                                                           count      mean   
Prim_NOI_Dsc                                                                 
Dermatitis                                         4586           0.960096   
Inflammation                                       21784          0.792325   
Puncture                                           13835          0.912685   
Multiple Physical Injuries                         8576           0.714552   
Injury                                             9762           0.782217   
Respiratory Disorders                              2269           0.840899   
Syncope                                            12590          0.943288   
Infection                                          1875           0.820267   
Strain                                             211564         0.679355   
Foreign body                                       8312           0.955245   
Multiple Injuries                                  1480           0.658784   
Chemical poisoning                                 664            0.823795   
Burn                                               7523           0.844876   
General Poisoning                                  145            0.882759   
Laceration                                         52728          0.865897   
Exposure to dust                                   205            0.751220   
Cumulative injury                                  4658           0.535208   
Asphyxiation                                       88             0.909091   
Angina pectoris (chest pain)                       1623           0.927911   
Mental stress                                      6195           0.790476   
Rupture                                            229            0.572052   
Amputation                                         1000           0.202000   
Occupational disease                               342            0.634503   
No Physical Injury                                 59             0.898305   
Fracture                                           24608          0.419660   
Contusion                                          84735          0.801227   
Vascular                                           670            0.883582   
Aids                                               6              0.833333   
Asbestosis (lung disease from breathing asbestos)  1881           0.699096   
Black Lung disease                                 212            0.698113   

                                                  deEscInd3000_y            
                                                           count      mean  
Prim_NOI_Dsc                                                                
Dermatitis                                         590.0          0.950847  
Inflammation                                       457.0          0.958425  
Puncture                                           102.0          0.960784  
Multiple Physical Injuries                         86.0           0.965116  
Injury                                             82.0           0.939024  
Respiratory Disorders                              72.0           0.861111  
Syncope                                            34.0           0.941176  
Infection                                          17.0           0.941176  
Strain                                             14.0           1.000000  
Foreign body                                       13.0           1.000000  
Multiple Injuries                                  11.0           0.909091  
Chemical poisoning                                 10.0           0.900000  
Burn                                               10.0           1.000000  
General Poisoning                                  9.0            0.888889  
Laceration                                         8.0            0.875000  
Exposure to dust                                   7.0            1.000000  
Cumulative i

**Comments:** not solely related to any NOI or POB

#### Seizure Indicator

In [48]:
szur_idx = acct_tfidf1[:,acct_features1.index('seizure')]. nonzero()

In [49]:
df.iloc[szur_idx][[ 'Claimant_Age_Qty',
                   'current_total_incurred',
                   'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred  deEscInd3000
count  2304.000000       2304.000000             2304.000000 
mean   34.343316         2537.440104             0.942274    
std    12.521236         27492.765608            0.233275    
min    15.000000         0.000000                0.000000    
25%    24.000000         0.000000                1.000000    
50%    31.000000         0.000000                1.000000    
75%    43.000000         0.000000                1.000000    
max    73.000000         902644.000000           1.000000

In [50]:
df['szur_idx'] = np.where(acct_tfidf1[:,acct_features1.index('seizure')] > 0, 1, 0)

In [52]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'szur_idx', 
             ['Prim_NOI_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x  \
                                                                       count   
Prim_NOI_Dsc                                                                   
Syncope                                                        12590           
Contusion                                                      84735           
Laceration                                                     52728           
Injury                                                         9762            
Vascular                                                       670             
Strain                                                         211564          
Multiple Physical Injuries                                     8576            
Inflammation                                                   21784           
Fracture                                                       24608           
Concussion                                                     5585            
Mental stress                                                  6195            
Dislocation                                                    2289            
Multiple Injuries                                              1480            
Sprain                                                         32230           
Puncture                                                       13835           
Contagious diseases                                            789             
Foreign body                                                   8312            
Burn                                                           7523            
Angina pectoris (chest pain)                                   1623            
Cumulative injury                                              4658            
Myocardial infarction (heart attack)                           2202            
Occupational disease                                           342             
Heat prostration (heat stroke)                                 282             
Respiratory Disorders                                          2269            
Dermatitis                                                     4586            
Carpal tunnel syndrome                                         2723            
Aids                                                           6               
Amputation                                                     1000            
Asbestosis (lung disease from breathing asbestos)              1881            
Asphyxiation                                                   88              
Black Lung disease                                             212             
Byssinosis (lung disease from breathing cotton, flax or hemp)  3               
Cancer                                                         160             
Chemical poisoning                                             664             
Crushing                                                       2905            
Electric Shock                                                 711             
Enucleation (surgical removal of a tumor, eye, etc.)           12              
Exposure to dust                                               205             
Freezing                                                       98              
General Poisoning                                              145             
Hernia                                                         6462            
Infection                                                      1875            
Loss of hearing                                                2135            
Mental disorder                                                52              
Metal poisoning                                                32              
No Physical Injury                                             59              
Radiation                                                      28              
Rupture                                                       

In [53]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'szur_idx', 
             ['Occur_Dsc'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x  \
                                                                               count   
Occur_Dsc                                                                              
other than physical cause of injury                                    19313           
Cumulative, not otherwise classified                                   12892           
Other - miscellaneous, not otherwise classified                        13137           
fall/slip/trip on same level                                           25274           
fall, slip or trip, not otherwise classified                           19272           
strain or injury by not otherwise specified                            71699           
Striking Against or Stepping On Stationary Object                      11148           
Injury by Person - fellow worker/patient (not in an act of crime)      8724            
fall/slip/trip from different level/ elevation                         10971           
Collision with a fixed object                                          1171            
Striking Against or Stepping On, Not Otherwise Classified              7206            
Absorption, ingestion, or inhalation not otherwise specified           11084           
Motor vehicle, not otherwise classified                                4144            
Fall/slip/trip on stairs                                               7812            
Collision or sideswipe with another vehicle                            10885           
Burn or Scald by Temperature extremes                                  932             
Strain or injury by holding or carrying                                6296            
cut, puncture, scrape not otherwise classified                         12991           
Miscellaneous                                                          12805           
fall/slip/trip from ladder or scaffolding                              5746            
Strain or injury by lifting or carrying                                57549           
fall/slip/trip into openings                                           1530            
fall/slip/trip from liquid or grease spills                            9284            
Injury from Falling or flying object                                   27555           
Injury from Motor vehicle                                              2310            
Cut/puncture/scrape by hand tool, utensil, not powered                 9532            
Vehicle upset (overturned)                                             1196            
Contact with animal or insect                                          8146            
Cut/puncture/scrape by broken glass                                    2665            
Strain or injury by repetitive motion                                  25881           
...                                                                     ...            
Cut/puncture/scrape by powered hand tool, appliances                   3349            
Explosion or flare back                                                233             
Injury from Hand Tool or Machine in Use                                2899            
Injury from Moving Parts of Machine                                    987             
Injury from Object Being Lifted or Handled                             11167           
Injury from Object Handled by Others                                   2931            
Strain or injury by continual noise                                    2247            
Strain or injury by jumping or leaping                                 1130            
Strain or injury by reaching                                           4841            
Strain or injury by twisting                                           6535            
Strain or injury by using tool or machinery                            3430            
Strain or injury by wielding or throwing                               430             
Striking Against or St

In [61]:
df['Current_Controverted_I'] = df['Current_Controverted_I'].fillna('N')

In [62]:
seg_analysis(df, 
             ['deEscInd3000'], 
             'szur_idx', 
             ['Current_Controverted_I'],
             ['count', 'mean']).sort_values(by=[('deEscInd3000_y', 'count')], ascending = False)

deEscInd3000_x           deEscInd3000_y          
                                count      mean          count      mean
Current_Controverted_I                                                  
Y                       72107          0.697200  1153           0.965308
N                       472422         0.734523  1151           0.919201

**Comment:** Seizure are denied 50%. Check why some of them are not denied

In [71]:
df[(df['szur_idx'] == 1) & (df['Current_Controverted_I'] == 'N')][['Occur_Dsc','deEscInd3000']].groupby(['Occur_Dsc']).agg(['count'])

deEscInd3000
                                                                         count
Occur_Dsc                                                                     
Absorption, ingestion, or inhalation not otherwise specified       4          
Burn or Scald by Temperature extremes                              1          
Collision or sideswipe with another vehicle                        3          
Collision with a fixed object                                      4          
Cumulative, not otherwise classified                               290        
Fall/slip/trip on stairs                                           3          
Injury by Person - fellow worker/patient (not in an act of crime)  1          
Motor vehicle, not otherwise classified                            4          
Other - miscellaneous, not otherwise classified                    179        
Striking Against or Stepping On Stationary Object                  9          
Striking Against or Stepping On, Not Otherwise Classified          3          
Vehicle upset (overturned)                                         1          
cut, puncture, scrape not otherwise classified                     4          
fall, slip or trip, not otherwise classified                       44         
fall/slip/trip from different level/ elevation                     8          
fall/slip/trip from ladder or scaffolding                          1          
fall/slip/trip into openings                                       2          
fall/slip/trip on same level                                       51         
other than physical cause of injury                                509        
slip or trip, did not fall                                         1          
strain or injury by not otherwise specified                        31

In [57]:
# 'feeling well' 861 count 0.94
# 'well' 1759 0.86
# 'went home' 446 count 0.83
test_idx = acct_tfidf2[:,acct_features2.index('feeling well')]. nonzero()

In [58]:
df.iloc[test_idx][[ 'Claimant_Age_Qty',
                   'current_total_incurred',
                   'deEscInd3000']].describe()

Claimant_Age_Qty  current_total_incurred  deEscInd3000
count  861.000000        8.610000e+02            861.000000  
mean   42.454123         3.946842e+03            0.948897    
std    15.249759         7.820282e+04            0.220337    
min    16.000000         0.000000e+00            0.000000    
25%    29.000000         0.000000e+00            1.000000    
50%    43.000000         0.000000e+00            1.000000    
75%    55.000000         0.000000e+00            1.000000    
max    80.000000         2.280347e+06            1.000000